In [ ]:
using Polyhedra
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector

In [ ]:
import MechanismComplementarity

In [ ]:
using Plots; gr()

In [ ]:
using JuMP, Gurobi

In [ ]:
reload("MechanismComplementarity")
mc = MechanismComplementarity

In [ ]:
g = -9.81
world = RigidBody{Float64}("world")
hopper = Mechanism(world; gravity=SVector(0, 0, g))

frame = CartesianFrame3D("core")
mass = 1.
inertia = SpatialInertia(frame, 0.01 * eye(SMatrix{3, 3}), SVector(0., 0, 0), mass)
core = RigidBody(inertia)

floating_base = Joint("core_to_world", QuaternionFloating{Float64}())
attach!(hopper, world, floating_base, eye(Transform3D, frame_before(floating_base), default_frame(world)), core)

frame = CartesianFrame3D("foot")
mass = 1.
inertia = SpatialInertia(frame, 0.01 * eye(SMatrix{3, 3}), SVector(0., 0, 0), mass)
foot = RigidBody(inertia)
leg = Joint("leg", Prismatic(SVector(0, 0, -1.)))
attach!(hopper, core, leg, eye(Transform3D, frame_before(leg), default_frame(core)), foot)

foot_contact = Point3D(default_frame(foot), SVector(0., 0, -0.1))

In [ ]:
vis = Visualizer()[:hopper]

setgeometry!(vis, hopper, Dict(
        default_frame(core) => [HyperSphere(Point(0., 0, 0), 0.15)],
        default_frame(foot) => [HyperSphere(Point(0., 0, 0), 0.1)]
    ))

In [ ]:
env = mc.Environment(
    Dict(foot => mc.ContactEnvironment(
            [foot_contact],
            [
                mc.Obstacle(
                    default_frame(world),
                    SimpleHRepresentation{3, Float64}([0 0 1], [0]),
                    HalfSpace{3, Float64}([0, 0, 1], 0),
                    0.5
                ),
            ],
            [
                mc.FreeRegion(
                    default_frame(world),
                    SimpleHRepresentation{3, Float64}(
                        [0 0 -1],
                        [0])
                )
            ])
        )
)

limits = Dict(
    leg => SimpleHRepresentation{1, Float64}([1 -1]', [1.05, -0.5])
)

In [ ]:
x0 = MechanismState(hopper, [0., 0, 0, 1, 0, 0, 1.5, 1.0], zeros(num_velocities(hopper)))

controller = (x) -> begin
    kp = 50
    kd = 0.1 * kp
    u = zeros(num_velocities(x))
    u[end] = kp * (1 - configuration(x)[end]) - kd * velocity(x)[end]
    u
end

Δt = 0.05
results = mc.simulate(x0, controller, limits, env, Δt, 20, GurobiSolver(OutputFlag=0));

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
plot([configuration(r.state, leg)[1] for r in results])

In [ ]:
Δt = 0.05
x0 = MechanismState(hopper, [0., 0, 0, 1, 0, 0, 1.0, 1.0], zeros(num_velocities(hopper)))
input_limits = Dict(leg=>[-25, 25])

model, results_opt = mc.optimize(x0, input_limits, limits, env, Δt, 20, Model(solver=GurobiSolver()))
@objective model Max configuration(results_opt[end].state, floating_base)[end]
for i in 1:length(results_opt)
    @constraint(model, results_opt[i].state.q[1:6] .== [0, 0, 0, 1, 0, 0])
end
solve(model)
results_opt = getvalue.(results_opt);

In [ ]:
for r in results_opt
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
plot([configuration(r.state, leg)[1] for r in results_opt])